<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/augmentation/documented_augmentation/amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Obbiettivo
Crea un dataset con le stesse dimensioni e proporzioni di quello riginale ma effettuando una sostituzione con dei sinonimi o con delle parole semanticamnete simili.

## **Import vari e inizializzazione pacchetti**

In [ ]:
pip install textattack

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **NOTE IMPORTANTI**


> la funzione **Augmenter.augment(arg)** utilizza al suo interno **random**, peccato che non si puo settare il seed durante la fase di costruzione dell'oggetto o quando viene richiamata la funzione, questo ha comportato l'import di random per forzare il seed prima dell'esecuzione del codice.



In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import timeit # per il testing

# **AMAZON**


In [7]:
dataset = pd.read_csv('/content/drive/Shareddrives/Deep Learning/datasets/Amazon/Reviews.csv')
print(f"COLONNE:{dataset.columns}")
print(f"DIMENSIONE:{dataset.shape}")

COLONNE:Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')
DIMENSIONE:(568454, 10)


Filtering del dataset dei dati duplicati e rimozione parti inutili di esso

In [8]:
rar=dataset.copy()
#print(rar.groupby(['Text']).count().shape)
rar.drop_duplicates(subset ="Text", 
                     keep = "last", inplace = True) 
rar.drop(['Id','ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator','HelpfulnessDenominator','Time'], axis=1, inplace=True)
print(f"COLONNE:{rar.columns}")
print(f"DIMENSIONE:{rar.shape}")

COLONNE:Index(['Score', 'Summary', 'Text'], dtype='object')
DIMENSIONE:(393579, 3)


**WordNet Augmenting**

In [14]:
from textattack.augmentation import WordNetAugmenter
from collections import OrderedDict
random.seed(33)

my_items = list()
number_of_trasformation_to_add=1 #raddoppia il dataset

start_time = timeit.default_timer()
augmenter = WordNetAugmenter(pct_words_to_swap=1, #percentuale di parolo sostituite 
                             transformations_per_example=number_of_trasformation_to_add)

for row in rar.head(10).itertuples():
  #print(row.Text)
  result=augmenter.augment(row.Text)
  # print(f'original:{s} \n')
  #print(f'Augmented:{result[0]} \n')
  #Aggiungo al dataset finale la entry originale del dataset
  my_items.append({'Score': row.Score,'Summary':row.Summary,'Text':row.Text})
  for i in range(number_of_trasformation_to_add):
    #Aggiungo al dataset finale una nuova entry augmentata 
    my_items.append({'Score': row.Score,'Summary':row.Summary,'Text':result[i]})
  

elapsed = timeit.default_timer() - start_time
print(f'TIME OF EXECUTION={elapsed}')
augmeted_new = pd.DataFrame(data=my_items, columns=rar.columns)
augmeted_new.to_pickle("Amazon_wordnet_sinonimi.pkl")
print(f"COLONNE:{augmeted_new.columns}")
print(f"DIMENSIONE:{augmeted_new.shape}")

TIME OF EXECUTION=10.000544521999927
COLONNE:Index(['Score', 'Summary', 'Text'], dtype='object')
DIMENSIONE:(20, 3)


**Embedding Augmenting**

In [ ]:
# import transformations, contraints, and the Augmenter
from textattack.augmentation import EmbeddingAugmenter
#Set the seed or the augmenter
random.seed(33)

my_items = list()
number_of_trasformation_to_add=1

start_time = timeit.default_timer()
augmenter = EmbeddingAugmenter(pct_words_to_swap=1, transformations_per_example=number_of_trasformation_to_add)
for row in rar.head(10).itertuples():
  #print(row.Text)
  result=augmenter.augment(row.Text)
  # print(f'original:{s} \n')
  #print(f'Augmented:{result[0]} \n')
  #Aggiungo al dataset finale la entry originale del dataset
  my_items.append({'Score': row.Score,'Summary':row.Summary,'Text':row.Text})
  for i in range(number_of_trasformation_to_add):
    #Aggiungo al dataset finale una nuova entry augmentata 
    my_items.append({'Score': row.Score,'Summary':row.Summary,'Text':result[i]})

elapsed = timeit.default_timer() - start_time
print(f'TIME OF EXECUTION={elapsed}')
augmeted_new = pd.DataFrame(data=my_items, columns=rar.columns)
print(f"COLONNE:{augmeted_new.columns}")
print(f"DIMENSIONE:{augmeted_new.shape}")

TIME OF EXECUTION=115.54404126700001
COLONNE:Index(['Score', 'Summary', 'Text'], dtype='object')
DIMENSIONE:(30, 3)


In [ ]:
pd.set_option('display.max_colwidth', -1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [ ]:
augmeted_new.head(3).Text

0    I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.                           
1    I have acquiring differing of the Dynamism tinned doggy meals product and have detected them all to be of alright quality. The merchandise seems more adores a simmer than a treated carnes and it olfactory best. My Labrador is fastidious and she grateful this commodities optimum than  most.
2    I have purchase myriad of the Dynamism tinned terrier catering merchandise and have discovered them all to be of alright quality. The commodity seems more loves a simmer than a process meats and it reeks optimum. My Labrador is fastidious and she grateful this commodities best than  most. 
Name: Text, dtype: object